In [248]:
import numpy as np
import pandas as pd
from lxml import html # Tool for webscraping
import requests # HTTP library to load wiki page
import re
from geopy.geocoders import Nominatim 

#### Making the request and scraping the data

In this part I used __lxml__ to scrape the data from the wiki webpage, I made this request using the __requests__ library

In [25]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree = html.document_fromstring(page.content)
data = tree.cssselect('.wikitable')
table = html.document_fromstring(data[0].text_content())



In [336]:
postal_info = list(filter(None, data[0].text_content().split('\n'))) # Spliting the raw table data to get a list

In [337]:
postal_info_data = postal_info[3:] # Get rid the first row

#### Creating the matrix

Using reshape on a numpy array that has a list with the Neighborhoods data

In [338]:
list_of_neigh = pd.DataFrame(np.array(postal_info_data).reshape((289,3)))

In [339]:
list_of_neigh = list_of_neigh.rename(columns={0:'PostalCode', 1:'Borough', 2: 'Neighborhood'}) # Adding column names

#### Convertign 'Not assigned' to NaN

To later use dataframe.dropna() to remove those unwanted rows

In [340]:
def cleanData(x):
    if(x == 'Not assigned'):
        return np.nan
    else:
        return x

list_of_neigh['Borough'] = list_of_neigh['Borough'].apply(lambda x: cleanData(x))

Removing all the rows with NaN 

In [341]:
list_of_neigh = list_of_neigh.dropna()
list_of_neigh = list_of_neigh.reset_index(drop=True)

In [354]:
postal_codes = list_of_neigh.groupby('Borough')['PostalCode'].unique()
#postal_codes['Central Toronto']

In [369]:
boroughs = list_of_neigh.groupby('Borough')['Neighborhood'].apply(lambda tags: ','.join(tags))
df_boroughs = pd.DataFrame(boroughs).reset_index()
#postal_codes.apply(lambda x: print(x))
df_boroughs['PostalCode'] = np.nan

for ind, item in enumerate(postal_codes):
    df_boroughs['PostalCode'][ind] = postal_codes[ind][0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# Changes the order of columns
reorder = [df_boroughs.columns[-1]] + list(df_boroughs.columns[:-1])

In [382]:
df_boroughs = df_boroughs[reorder]
# Set 'Not assigned' to the neighborhood borough
for ind, item in enumerate(df_boroughs['Neighborhood']):
    if(item == 'Not assigned'):
        df_boroughs['Neighborhood'][ind] = df_boroughs['Borough'][ind]
    
df_boroughs

,PostalCode,Borough,Neighborhood
0,M4N,Central Toronto,"Lawrence Park,Roselawn,Davisville North,Forest..."
1,M5A,Downtown Toronto,"Harbourfront,Regent Park,Ryerson,Garden Distri..."
2,M4E,East Toronto,"The Beaches,The Danforth West,Riverdale,The Be..."
3,M4B,East York,"Woodbine Gardens,Parkview Hill,Woodbine Height..."
4,M9A,Etobicoke,"Islington Avenue,Cloverdale,Islington,Martin G..."
5,M7R,Mississauga,Canada Post Gateway Processing Centre
6,M3A,North York,"Parkwoods,Victoria Village,Lawrence Heights,La..."
7,M7A,Queen's Park,Queen's Park
8,M1B,Scarborough,"Rouge,Malvern,Highland Creek,Rouge Hill,Port U..."
9,M6H,West Toronto,"Dovercourt Village,Dufferin,Little Portugal,Tr..."


In [383]:
df_boroughs.shape

(11, 3)